In [ ]:
#!pip install mediapipe

In [ ]:
#Modelo pre-entrenado
import mediapipe as mp
#Lector de imágenes
import cv2
#Manejo de datos
import numpy as np
#Lector de direcciones
import os

#Mezcla
from sklearn.utils import shuffle
#División de datos
from sklearn.model_selection import train_test_split
#Modelo a entrenar
from sklearn.neighbors import KNeighborsClassifier
#Captura de resultados
from sklearn import metrics

#Impresión de tablas
import matplotlib.pyplot as plt

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [ ]:
def get_param(Class):
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
        data=[]
        for idx, file in enumerate(os.listdir(Class)):
            # Read an image, flip it around y-axis for correct handedness output (see
            # above).
            image = cv2.flip(cv2.imread(os.path.join(Class,file)), 1)
            # Convert the BGR image to RGB before processing.
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if not results.multi_hand_landmarks:
                continue
            image_height, image_width, _ = image.shape
            annotated_image = image.copy()

            for hand_landmarks in results.multi_hand_landmarks:
                points=[]
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST])

                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP])

                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP])

                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP])

                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP])

                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP])
                points.append(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP])

                coor=[]
                for i in enumerate(points):
                    coor.append(i[1].x)
                    coor.append(i[1].y)
                    coor.append(i[1].z)
                data.append([coor,Class])
    return data

In [ ]:
def xy_capture(data):
    X=[]
    Y=[]
    for features,labels in data:
        X.append(features)
        Y.append(labels)
    return X,Y

In [ ]:
#Captura de características
dataa = get_param('A')
datae = get_param('E')
datai = get_param('I')
datao = get_param('O')
datau = get_param('U')

#Unión y mezcla de datos
data=np.concatenate((dataa,datae,datai,datao,datau))
data = shuffle(data)

#Separación de datos
train_data,test_data = train_test_split(data, test_size=0.2)
X_train,Y_train = xy_capture(train_data)
X_test,Y_test = xy_capture(test_data)

In [ ]:
#para matrix de confusion
from sklearn.metrics import confusion_matrix

#para muestra de matrix de confusion
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
model = KNeighborsClassifier(n_neighbors=7)
model.fit(X_train, Y_train)
predicted_knn = model.predict(X_test)

In [ ]:
#Resultados
print(
    f"Classification report for classifier {model}:\n"
    f"{metrics.classification_report(Y_test, predicted_knn)}\n"
)

arr = confusion_matrix(Y_test, predicted_knn)

df_cm = pd.DataFrame(arr, index = [i for i in "AEIOU"],columns = [i for i in "AEIOU"])
plt.figure(figsize = (6,4))
sn.heatmap(df_cm, annot=True)
plt.show()

In [ ]:
# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    max_num_hands=1,
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        
        points=[]
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST])

        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_CMC])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP])

        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP])

        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP])

        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_DIP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP])

        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_DIP])
        points.append(hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP])
        
        data=[]
        coor=[]
        for i in enumerate(points):
            coor.append(i[1].x)
            coor.append(i[1].y)
            coor.append(i[1].z)
        data.append(coor)
        
        clase = model.predict(data)
        
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # Flip the image horizontally for a selfie-view display.
    image = cv2.flip(image, 1)
    image = cv2.putText(img = image,text = str(clase),org = (200, 200),fontFace = cv2.FONT_HERSHEY_DUPLEX,
                        fontScale = 1.0,color = (125, 246, 55),thickness = 3)
    
    cv2.imshow('MediaPipe Hands', image)
    cv2.waitKey(5)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()